In [ ]:
# 当我们拿到一批原始的数据

# 1.首先要明确有多少特征，哪些是连续的，哪些是类别的。
# 2.检查有没有缺失值，对确实的特征选择恰当方式进行弥补，使数据完整。
# 3.对连续的数值型特征进行标准化，使得均值为0，方差为1。
# 4.对类别型的特征进行one-hot编码。
# 5.将需要转换成类别型数据的连续型数据进行二值化。
# 6.为防止过拟合或者其他原因，选择是否要将数据进行正则化。
# 7.在对数据进行初探之后发现效果不佳，可以尝试使用多项式方法，寻找非线性的关系。
# 8.根据实际问题分析是否需要对特征进行相应的函数转换。

In [15]:
#加载数据
import numpy as np
from sklearn import model_selection
#新版的库将cross_validation 改为model_selection
# from sklearn.model_selection import validation_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn import datasets
from sklearn import svm

iris  = datasets.load_iris()
type(iris.data),iris.target.shape

(numpy.ndarray, (150,))

In [16]:
# 预处理数据的方法总结（使用sklearn-preprocessing）
from sklearn import preprocessing
import numpy as np
# 1. 标准化：去均值，方差规模化
 
# 创建一组特征数据，每一行表示一个样本，每一列表示一个特征
# Standardization标准化:将特征数据的分布调整成标准正太分布，也叫高斯分布，也就是使得数据的均值维0，方差为1.
# 标准化的原因在于如果有些特征的方差过大，则会主导目标函数从而使参数估计器无法正确地去学习其他特征。
# 标准化的过程为两步：去均值的中心化（均值变为0）；方差的规模化（方差变为1）。
# 在sklearn.preprocessing中提供了一个scale的方法，可以实现以上功能。
x = np.array([[1., -1., 2.],
              [8., 0., 0.],
              [0., 1., -1.]])
# 将每一列特征标准化为标准正太分布，注意，标准化是针对每一列而言的
x_scale = preprocessing.scale(x)
x_scale,type(x)

(array([[-0.56195149, -1.22474487,  1.33630621],
        [ 1.40487872,  0.        , -0.26726124],
        [-0.84292723,  1.22474487, -1.06904497]]), numpy.ndarray)

In [12]:
# 可以查看标准化后的数据的均值与方差，已经变成0,1了
# axis=0 表示对每一列
x_scale.mean(axis=0)

array([ 0.,  0.,  0.])

In [13]:
# 同理，看一下标准差
x_scale.std(axis=0)

array([ 1.,  1.,  1.])

In [17]:
# preprocessing这个模块还提供了一个实用类StandarScaler，它可以在训练数据集上做了标准转换操作之后，把相同的转换应用到测试训练集中。
# 这是相当好的一个功能。可以对训练数据，测试数据应用相同的转换，以后有新的数据进来也可以直接调用，不用再重新把数据放在一起再计算一次了。
# 调用fit方法，根据已有的训练数据创建一个标准化的转换器
# 另外，StandardScaler()中可以传入两个参数：with_mean,with_std.这两个都是布尔型的参数，
# 默认情况下都是true,但也可以自定义成false.即不要均值中心化或者不要方差规模化为1.
scaler = preprocessing.StandardScaler().fit(x)
 
scaler

StandardScaler(copy=True, with_mean=True, with_std=True)

In [30]:
# 使用上面这个转换器去转换训练数据x,调用transform方法
old  =scaler.transform(x)
########################################
# 好了，比如现在又来了一组新的样本，也想得到相同的转换
new_x = [[-1., 1., 0.]]
new  =scaler.transform(new_x)
new
##################################

array([[-1.12390297,  1.22474487, -0.26726124]])

In [23]:
new.mean(axis=0)

array([-1.12390297,  1.22474487, -0.26726124])

In [25]:
old.mean(axis= 0)#  对测试集进行相同的标准化操作后 不一定会获得均值为一的数

array([ 0.,  0.,  0.])

In [26]:
# 2.MinMaxScaler
# 在MinMaxScaler中是给定了一个明确的最大值与最小值。它的计算公式如下：
# X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
# X_scaled = X_std / (max - min) + min
# 以下这个例子是将数据规与[0,1]之间，每个特征中的最小值变成了0，最大值变成了1，请看：
min_max_scaler = preprocessing.MinMaxScaler()
x_minmax = min_max_scaler.fit_transform(x)
x_minmax
################################################################################
# 同样的，如果有新的测试数据进来，也想做同样的转换咋办呢？请看：
x_test = np.array([[-3., -1., 4.]])
x_test_minmax = min_max_scaler.transform(x_test)
x_test_minmax


array([[-0.375     ,  0.        ,  1.66666667]])

In [32]:
new  =np.array( [[1.,2.,3.]])
new = min_max_scaler.transform(new)
new


array([[ 0.125     ,  1.5       ,  1.33333333]])

In [34]:
#以上都是对数据进行标准化 中心化 通过构造一个scaler来进行统一操作
#3.同理构造一个正则器来进行正则
# preprocessing这个模块还提供了一个实用类Normalizer,实用transform方法同样也可以对新的数据进行同样的转换
# 根据训练数据创建一个正则器
normalizer = preprocessing.Normalizer().fit(x)
print(normalizer,111)
# 对训练数据进行正则
print(normalizer.transform(x),222)
###################################################################
# 对新的测试数据进行正则
normalizer.transform([[-1., 1., 0.]])
# normalize和Normalizer都既可以用在密集数组也可以用在稀疏矩阵（scipy.sparse)中
# 对于稀疏的输入数据，它会被转变成维亚索的稀疏行表征（具体请见scipy.sparse.csr_matrix)
###################################################################################

Normalizer(copy=True, norm='l2') 111
[[ 0.40824829 -0.40824829  0.81649658]
 [ 1.          0.          0.        ]
 [ 0.          0.70710678 -0.70710678]] 222


array([[-0.70710678,  0.70710678,  0.        ]])

In [37]:
#4.二值化
# 3 二值化–特征的二值化
# 特征的二值化是指将数值型的特征数据转换成布尔类型的值。可以使用实用类Binarizer
# 默认是根据0来二值化，大于0的都标记为1，小于等于0的都标记为0。
from sklearn import preprocessing
import numpy as np
 
# 创建一组特征数据，每一行表示一个样本，每一列表示一个特征
x = np.array([[1., -1., 2.],
              [2., 0., 0.],
              [0., 1., -1.]])
 
binarizer = preprocessing.Binarizer().fit(x)
binarizer.transform(x)
#################################################################################
#当然也可以自己设置这个阀值，超过多少认为是1，只需传出参数threshold即可
binarizer = preprocessing.Binarizer(threshold=1.5)
binarizer.transform(x)
##########################################################################3

array([[ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       [ 0.,  0.,  0.]])

In [86]:
#5.标签编码
import sklearn
le = sklearn.preprocessing.LabelEncoder()
le.fit([1, 2, 2, 6])
le.transform([1, 1, 2, 6]) #array([0, 0, 1, 2])
#非数值型转化为数值型
le.fit(["paris", "paris", "tokyo", "amsterdam"])
le.transform(["tokyo", "tokyo", "paris"]) #array([2, 2, 1])

array([2, 2, 1], dtype=int64)

In [42]:
#6..OneHotEncoder独热编码
#独热编码和标签编码 前者都是用01来表示，并且任意两个变量在向量空间距离都相等。标签编码则用1~n来表示n个离散值，这样每个特征会不平等
#可以结合二者  先将数据标签化 再独热
from sklearn import preprocessing
enc = preprocessing.OneHotEncoder(categories='auto')
enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])    # fit来学习编码
enc.transform([[0, 1, 3]]).toarray()    # 进行编码
#fit进去的有4*3的数据，特征有3为，第一维有 0 1 两种值所以独热为
#独热编码（哑变量 dummy variable）是因为大部分算法是基于向量空间中的度量来进行计算的，为了使非偏序关系的变量取值不具有偏序性，
#并且到圆点是等距的。使用one-hot编码，将离散特征的取值扩展到了欧式空间，离散特征的某个取值就对应欧式空间的某个点。
#将离散型特征使用one-hot编码，会让特征之间的距离计算更加合理。
#离散特征进行one-hot编码后，编码后的特征，其实每一维度的特征都可以看做是连续的特征。
#就可以跟对连续型特征的归一化方法一样，对每一维特征进行归一化。比如归一化到[-1,1]或归一化到均值为0,方差为1


array([[ 1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.]])

In [72]:
# 为类别特征编码
# 我们知道特征可能是连续型的也可能是类别型的变量，比如说：
# [“male”, “female”], [“from Europe”, “from US”, “from Asia”], [“uses Firefox”, “uses Chrome”, “uses Safari”, “uses Internet Explorer”].
# 这些类别特征无法直接进入模型，它们需要被转换成整数来表征，比如：
# [“male”, “from US”, “uses Internet Explorer”] could be expressed as [0, 1, 3] while [“female”, “from Asia”, “uses Chrome”] would be [1, 2, 1].
# 然而上面这种表征的方式仍然不能直接为scikit-learn的模型所用，因为模型会把它们当成序列型的连续变量。
# 要想使得类别型的变量能最终被模型直接使用，可以使用one-of-k编码或者one-hot编码。
# 这些都可以通过OneHotEncoder实现，它可以将有n种值的一个特征变成n个二元的特征。
# 特征1中有(0,1）两个值，特征2中有(0,1,2)3个值，特征3中有（0,1,2,3)4个值，所以编码之后总共有9个二元特征。
 
enc = preprocessing.OneHotEncoder(categories='auto')#categories='auto'用于设置自动寻找该列的特征种类
enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])
enc.transform([[0,1,3]]).toarray()
##########################################################################
# 但是呢，也会存在这样的情况，某些特征中可能对一些值有缺失，
# 比如明明有男女两个性别，样本数据中都是男性，这样就会默认被判别为我只有一类值。
# 这个时候我们可以向OneHotEncoder传如参数n_values，用来指明每个特征中的值的总个数
enc = preprocessing.OneHotEncoder(n_values=[2,3,4])  # 指明每个特征中的值的总个数分别为 2 3 4
enc.fit([[1, 2, 3], [0, 2, 0]])
enc.transform([[1,0,0]]).toarray()
enc.categories_
#############################################################################

C:\Users\SBY\Anaconda3\envs\tensorflow1\lib\site-packages\sklearn\preprocessing\_encoders.py:331: DeprecationWarning: Passing 'n_values' is deprecated in version 0.20 and will be removed in 0.22. You can use the 'categories' keyword instead. 'n_values=n' corresponds to 'categories=[range(n)]'.
  warnings.warn(msg, DeprecationWarning)


[array([ 0.]), array([ 0.,  1.]), array([ 0.,  1.,  2.])]

In [ ]:
#缺点：当类别的数量很多时，特征空间会变得非常大。在这种情况下，一般可以用PCA来减少维度。
#而且one hot encoding+PCA这种组合在实际中也非常有用。

#将离散数据进行独热编码是为了更好计算向量距离，如果模型本身对距离不要求或者离散数据过多，不适合使用+

In [81]:
# 7.. 弥补缺失数据
# 在scikit-learn的模型中都是假设输入的数据是数值型的，并且都是有意义的，如果有缺失数据是通过NAN，或者空值表示的话，就无法识别与计算了。
# 要弥补缺失值，可以使用均值，中位数，众数等等。Imputer这个类可以实现。请看：
import numpy as np
from sklearn.preprocessing import Imputer
#from sklearn import impute.SimpleImputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit([[1, 20], [np.nan, 3], [7, 6]])
Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)
X = [[np.nan, 2], [6, np.nan], [7, 6]]
print(imp.transform(X))                           


[[ 4.          2.        ]
 [ 6.          9.66666667]
 [ 7.          6.        ]]


C:\Users\SBY\Anaconda3\envs\tensorflow1\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\SBY\Anaconda3\envs\tensorflow1\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [82]:
# Imputer类同样也可以支持稀疏矩阵,以下例子将0作为了缺失值，为其补上均值
import scipy.sparse as sp
 
# 创建一个稀疏矩阵
x = sp.csc_matrix([[1, 2], [0, 3], [7, 6]])
 
imp = Imputer(missing_values=0, strategy='mean', verbose=0)
imp.fit(x)
 
x_test = sp.csc_matrix([[0, 2], [6, 0], [7, 6]])
imp.transform(x_test)

C:\Users\SBY\Anaconda3\envs\tensorflow1\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


array([[ 4.        ,  2.        ],
       [ 6.        ,  3.66666667],
       [ 7.        ,  6.        ]])

In [83]:
# 8.. 创建多项式特征
# 有的时候线性的特征并不能做出美的模型，于是我们会去尝试非线性。非线性是建立在将特征进行多项式地展开上的。
 
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
 
# 自建一组3*2的样本
x = np.arange(6).reshape(3, 2)
# 创建2次方的多项式
# 比如将两个特征 (X_1, X_2)，它的平方展开式便转换成5个特征(1, X_1, X_2, X_1^2, X_1X_2, X_2^2). 代码案例如下：
poly = PolynomialFeatures(2)
poly.fit_transform(x)


array([[  1.,   0.,   1.,   0.,   0.,   1.],
       [  1.,   2.,   3.,   4.,   6.,   9.],
       [  1.,   4.,   5.,  16.,  20.,  25.]])

In [84]:
# 也可以自定义选择只要保留特征相乘的项。 
# 即将 (X_1, X_2, X_3) 转换成 (1, X_1, X_2, X_3, X_1X_2, X_1X_3, X_2X_3, X_1X_2X_3).
x = np.arange(9).reshape(3, 3)
 
poly = PolynomialFeatures(degree=3, interaction_only=True)
poly.fit_transform(x)

array([[   1.,    0.,    1.,    2.,    0.,    0.,    2.,    0.],
       [   1.,    3.,    4.,    5.,   12.,   15.,   20.,   60.],
       [   1.,    6.,    7.,    8.,   42.,   48.,   56.,  336.]])

In [ ]:
#9. 去除异常数据

In [89]:
#1. 自定义转换器
import numpy as np
from sklearn.preprocessing import FunctionTransformer
    
transformer = FunctionTransformer(np.log1p,validate=False)
X = np.array([[0, 1], [2, 3]])
transformer.transform(X)


array([[ 0.        ,  0.69314718],
       [ 1.09861229,  1.38629436]])